# Deposit in a periodic box

In [ ]:
from pathlib import Path

import numpy as np

from pylmgc90 import pre

datbox = Path('./DATBOX')
datbox.mkdir(exist_ok=True)

Creating container

In [ ]:
# 3D case
dim = 3

# containers creation
#   * for the bodies
bodies  = pre.avatars()
bodies2 = pre.avatars()
#   * for the materials
mats = pre.materials()
#   * for the models
mods = pre.models()
#   * for the visibility tables
svs = pre.see_tables()
#   * for the contact laws
tacts = pre.tact_behavs()

Creating materials and model

In [ ]:
# creation of two materials:
#   * for the spheres
plex = pre.material(name='PLEXx', materialType='RIGID', density=100.)
mats.addMaterial(plex)
#   * for the walls
tdur = pre.material(name='TDURx', materialType='RIGID', density=1000.)
mats.addMaterial(tdur)

# creation of rigid model in 3D
mod = pre.model(name='rigid', physics='MECAx', element='Rxx3D', dimension=dim)
mods.addModel(mod)

Generating particles

In [ ]:
# variables initialisation to deposit on a cubic grid:

# the number of particles to generate
nb_particles = 1000

# granulometry definition

# random distribution in [0.5, 2.[ 
#seed = list(range(33))
radii = pre.granulo_Random(nb_particles, 0.5, 2.)#, seed)

# minimum and maximum radii
radius_min = np.min(radii)
radius_max = np.max(radii)

Computing grains positions

In [ ]:
# deposit of particles under gravity load in a box
lx = 15.
ly = 15.
lz = 10.
nb_comp_particles, coor, radii = pre.depositInBox3D(radii, lx, ly, lz)

# warning if not all particles were deposited
if (nb_comp_particles < nb_particles):
    print("Warning: granulometry changed, since some particles cannot be deposited!")

# addition of spheres loop:
for r, c in zip(radii, coor):
    # new rigid sphere creation
    body = pre.rigidSphere(r=r, center=c, model=mod, material=plex, color='BLUEx')
    # addint the sphere to the avatar container
    bodies += body

# adding spheres of the inferior wall loop:
import itertools
X = np.arange(0., lx, radius_min)
Y = np.arange(0., ly, radius_min)
for x, y in itertools.product(X,Y):
    center=[x,y,-radius_min]
    # new rigid disk creation, made of plex material
    body2 = pre.rigidSphere(r=radius_min, center=center, 
                            model=mod, material=tdur, color='GREEN') 
    body2.imposeDrivenDof(component=[1,2,3,4,5,6], dofty='vlocy')
    # adding new body to the avatar container
    bodies2 += body2

bodies+=bodies2

Defining interaction laws and see tables

In [ ]:
# interactions management:
#   * law declaration
#       - between particles
lspsp = pre.tact_behav(name='iqsc0', law='IQS_CLB', fric=0.3)
tacts+= lspsp
#       - with the walls
lsppl = pre.tact_behav(name='iqsc1', law='IQS_CLB', fric=0.5)
tacts+= lsppl
#   * visibility tables declaration
#       - between particles
svspsp = pre.see_table(CorpsCandidat   ='RBDY3', candidat   ='SPHER', colorCandidat   ='BLUEx',
                       CorpsAntagoniste='RBDY3', antagoniste='SPHER', colorAntagoniste='BLUEx',
                       behav=lspsp, alert=0.1*radius_min)
svs+=svspsp
#       - with the walls
svsppl = pre.see_table(CorpsCandidat   ='RBDY3', candidat   ='SPHER', colorCandidat   ='BLUEx',
                       CorpsAntagoniste='RBDY3', antagoniste='SPHER', colorAntagoniste='GREEN',
                       behav=lsppl, alert=0.1*radius_min)
svs+=svsppl

Adding some post-processing commands

In [ ]:
post = pre.postpro_commands()

list_tracking = []
for i in range(1,8):
    list_tracking.append(bodies[i])

post.addCommand( pre.postpro_command(name='SOLVER INFORMATIONS', step=1) )
post.addCommand( pre.postpro_command(name='VIOLATION EVOLUTION', step=1) )
post.addCommand( pre.postpro_command(name='BODY TRACKING', step=1, rigid_set=list_tracking) )

Writing files

In [ ]:
# writting files
pre.writeDatbox(dim, mats, mods, bodies, tacts, svs, post=post, datbox_path=datbox)

Vizualisation of the sample 

In [ ]:
try:
    pre.visuAvatars(bodies)
except:
    pass

# Computation

In [ ]:
dim = 3

# time evolution parameters
dt = 1e-3
nb_steps = 500

# theta integrator parameter
theta = 0.5

# nlgs parameters
tol    = 1e-4
relax  = 1.0
norm   = 'Quad '
gs_it1 = 500
gs_it2 = 10
stype  = 'Stored_Delassus_Loops         '

# write parameter
freq_write   = 10

# display parameters
freq_display = 10

xperiode = 15.
yperiode = 15.

In [ ]:
from pylmgc90 import chipy
from pylmgc90.chipy import computation

computation.initialize(dim, dt, theta)

chipy.SetPeriodicCondition(xperiode,yperiode)

for k in range( 1, nb_steps+1 ):
    if k%50 == 0:
        print( f"computing step {k}" )
    computation.one_step(stype, norm, tol ,relax, gs_it1, gs_it2, freq_write, freq_display)
computation.finalize()

In [ ]:
!paraview